## Load Data Set

In [1]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical


print(tf.__version__)

root_path = './'
dataset_path = os.path.join(root_path, 'tf_dataset_gray')

models_path = os.path.join(root_path, 'saved_models_gray')
if not os.path.exists(models_path):
  os.mkdir(models_path)

Using TensorFlow backend.


1.13.1


In [2]:
BATCH_SIZE = 256


image_feature_description = {
    'enc': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
    'img': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True)
}


def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)


raw_train_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path,'train.tfrecords'))
parsed_train_dataset = raw_train_dataset.map(_parse_image_function)

# raw_val_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path, 'val.tfrecords'))
# parsed_val_dataset = raw_val_dataset.map(_parse_image_function)

# raw_test_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path, 'test.tfrecords'))
# parsed_test_dataset = raw_test_dataset.map(_parse_image_function)


parsed_train_dataset = parsed_train_dataset.repeat()
parsed_train_dataset = parsed_train_dataset.shuffle(6000)
parsed_train_dataset = parsed_train_dataset.batch(BATCH_SIZE)
dataset_iterator = parsed_train_dataset.make_one_shot_iterator()

variable_dataset = dataset_iterator.get_next()

## Wasserstein GAN

In [3]:
def d_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

## Model Design

In [4]:
enc_len = 0
age_len = 0
img_shape = (32, 32, 1)
width, height, depth = (32, 32, 1)
img_len = np.prod(img_shape)
latent_dim = enc_len + age_len + img_len
noise_len = 100
input_dim = enc_len + age_len + noise_len
cond_len = enc_len + age_len


def build_discriminator():
    conv = keras.Sequential([
        # conv block 1
        keras.layers.Conv2D(
            filters=16,
            kernel_size=(3, 3),
            strides=2,
            input_shape=img_shape
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        keras.layers.BatchNormalization(),

        # conv block 2
        keras.layers.Conv2D(
            filters=32,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        keras.layers.BatchNormalization(),
        
        # conv block 3
        keras.layers.Conv2D(
            filters=3,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
    ])
    
    conv.summary()
    
    model = keras.Sequential([
        # dense 1
#         keras.layers.Dense(128),
#         keras.layers.Activation(tf.nn.relu),
        
        # output
        keras.layers.Dense(1, input_shape=(27,)),
        keras.layers.Activation(tf.nn.sigmoid),
    ])
    
#     clf = keras.Sequential([
#         # dense 1
#         keras.layers.Dense(128, input_shape=(27+128+8,)),
#         keras.layers.Activation(tf.nn.relu),
        
#         # output
#         keras.layers.Dense(age_len),
#         keras.layers.Activation(tf.nn.softmax),
#     ])
    
    model.summary()
    
#     # condition
#     c1 = keras.layers.Input(shape=(enc_len,))
#     c2 = keras.layers.Input(shape=(age_len,))
    
    # image
    z = keras.layers.Input(shape=img_shape)
    
    # convolution
    zout = conv(z)
    
    # flatten image
    z_flat = keras.layers.Flatten()(zout)
    
    # concatenation
#     inputs = keras.layers.concatenate([z_flat])
    
    # real or fake
    outputs = model(z_flat)
    
    # age label
#     classes = clf(inputs)
    
    
    return keras.models.Model(z, outputs)


def build_generator():
    
    conv = keras.Sequential([
        # transpose conv block 1
        keras.layers.Conv2DTranspose(
            filters=16,
            kernel_size=(3, 3),
            strides=1,
            input_shape=(11, 11, 1)
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 2
        keras.layers.Conv2DTranspose(
            filters=32,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 3
        keras.layers.Conv2DTranspose(
            filters=32,
            kernel_size=(2, 2),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),

        # transpose conv block 4
        keras.layers.Conv2DTranspose(
            filters=1,
            kernel_size=(5, 5),
            strides=1
        ),
        
        # output
        keras.layers.Activation(tf.nn.tanh)
    ])
    
    conv.summary()
    
    model = keras.Sequential([
        # dense 1
        keras.layers.Dense(121, input_shape=(input_dim,)),
        keras.layers.Activation(tf.nn.relu),
        
        # reshape 1d to 3d
        keras.layers.Reshape((11, 11, 1))
    ])
    
    model.summary()
    
#     # condition
#     c1 = keras.layers.Input(shape=(enc_len,))
#     c2 = keras.layers.Input(shape=(age_len,))
    
    # noise
    x = keras.layers.Input(shape=(noise_len,))

    # concatenation
#     inputs = keras.layers.concatenate([x])
    
    # flat dense output
    out_1 = model(x)
    
    # transpose conv output
    outputs = conv(out_1)
    
    return keras.models.Model(x, outputs)


discriminator = build_discriminator()
generator = build_generator()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 15, 15, 16)        160       
_________________________________________________________________
activation_1 (Activation)    (None, 15, 15, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 15, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
activation_2 (Activation)    (None, 7, 7, 32)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 7, 32)          128       
_________________________________________________________________
conv

In [ ]:
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_4 (Sequential)    (None, 11, 11, 1)         12221     
_________________________________________________________________
sequential_3 (Sequential)    (None, 32, 32, 1)         10049     
Total params: 22,270
Trainable params: 22,110
Non-trainable params: 160
_________________________________________________________________


In [ ]:
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 3, 3, 3)           5859      
_________________________________________________________________
flatten_1 (Flatten)          (None, 27)                0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 28        
Total params: 5,887
Trainable params: 5,791
Non-trainable params: 96
_________________________________________________________________


## Generative Adversarial Network

In [ ]:
GLR = 0.01  # generator
DLR = 0.01  # discriminator


discriminator.compile(
    optimizer=keras.optimizers.Adam(DLR, 0.5),
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy']
)


# # condition
# c1 = keras.layers.Input(shape=(enc_len,))
# c2 = keras.layers.Input(shape=(age_len,))

# noise
x = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
z = generator(x)
out = discriminator(z)

# GAN
gan = keras.models.Model(inputs=x, outputs=out)

gan.compile(
    optimizer=keras.optimizers.Adam(GLR , 0.5),
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy'])

In [ ]:
gan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
model_2 (Model)              (None, 32, 32, 1)         22270     
_________________________________________________________________
model_1 (Model)              (None, 1)                 5887      
Total params: 28,157
Trainable params: 22,110
Non-trainable params: 6,047
_________________________________________________________________


## Visualization Method

In [ ]:
# from google.colab import drive
import os


# drive.mount('/content/gdrive', force_remount=True)

root_path = './'
tgt_pth = os.path.join(root_path, 'visualize_age-cgan-v7')

if not os.path.exists(tgt_pth):
  os.mkdir(tgt_pth)

In [ ]:
def visualizeGAN(e, z_real, z_fake):

    fig, axes = plt.subplots(4, 4, figsize=(20, 18))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0:
                cell.imshow(
                    np.squeeze(
                        0.5 * z_real[r_real * 4 + col] + 0.5,
                        axis=-1
                    ),
                    cmap='gray'
                )
            else:
                cell.imshow(
                    np.squeeze(
                        0.5 * z_fake[r_fake * 4 + col] + 0.5,
                        axis=-1
                    ),
                    cmap='gray'
                )

            cell.axis("off")

        if row % 2 == 0:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    plt.tight_layout()

    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))
    
    plt.close()

## Load Batch

In [ ]:
def load_noise():
    
    y_real = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_real = y_real.eval()

        while True:
            values = sess.run([variable_dataset])
            row = values[0]

            sz = row['img'].shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
            # concatenate face + age + noise
#             c1 = row['enc']
#             c2 = tf.cast(row['age'], tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            
            yield x, y_real


def load_batch():
    
    y_fake = tf.zeros((BATCH_SIZE,))
    y_real = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_fake = y_fake.eval()
        y_real = y_real.eval()

        while True:
            values = sess.run([variable_dataset])
            row = values[0]

            sz = row['img'].shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
#             c1 = row['enc']
#             c2 = tf.cast(row['age'], tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            z_fake = generator.predict(x)

            # real data
#             c1 = row['enc']
#             c2 = tf.cast(row['age'], tf.float32).eval()
            z_real = tf.reshape(tf.io.decode_raw(row['img'], tf.int64), (-1, width, height, depth))
    
            z_real = tf.cast(z_real, tf.float32)
    
            # scale to [-1, +1]
            z_real = tf.math.subtract(tf.math.divide(z_real, 127.5), 1)
        
            z_real = z_real.eval()
                        
            yield x, z_fake, y_fake, z_real, y_real

## Train Model

In [ ]:
EPOCHS = 10000
STEPS = 1  # 60000 // BATCH_SIZE


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


disc_itr = load_batch()
gen_itr = load_noise()


# epochs
for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS):
        
        x, z_fake, y_fake, z_real, y_real = next(disc_itr)

        # train
        loss_2, acc_2 = discriminator.train_on_batch(z_real, y_real)
        loss_1, acc_1 = discriminator.train_on_batch(z_fake, y_fake)

        batch_loss = 0.5 * (loss_1 + loss_2)
        batch_acc = 0.5 * (acc_1 + acc_2)

        loss.append(batch_loss)
        acc.append(batch_acc)

    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS):

        x, y_true = next(gen_itr)

        # train
        loss_1, acc_1 = gan.train_on_batch(x, y_true)

        loss.append(loss_1)
        acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("E: {}, [D ACC: %{:.2f}], [D LOSS: {:.2f}], [G ACC: %{:.2f}], [G LOSS: {:.2f}]".format(
          e,
          train_acc_d[-1] * 100,
          train_loss_d[-1] * 100,
          train_acc_g[-1] * 100,
          train_loss_g[-1] * 100
      ))

    if e % 100 == 0:
        ## visualize results
        x, z_fake, y_fake, z_real, y_real = next(disc_itr)
        visualizeGAN(e, z_real, z_fake)
        
        ## save model
        pth = os.path.join(models_path, 'gan.h5')
        gan.save(pth)

        pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
        generator.save(pth)

        pth = os.path.join(models_path, 'discriminator.h5')
        discriminator.save(pth)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


E: 0, [D ACC: %76.37], [D LOSS: 49.66], [G ACC: %69.53], [G LOSS: 59.09]
E: 1, [D ACC: %14.26], [D LOSS: 140.41], [G ACC: %20.70], [G LOSS: 129.79]
E: 2, [D ACC: %65.23], [D LOSS: 77.08], [G ACC: %3.12], [G LOSS: 225.74]
E: 3, [D ACC: %64.65], [D LOSS: 86.68], [G ACC: %7.03], [G LOSS: 199.39]
E: 4, [D ACC: %48.44], [D LOSS: 113.60], [G ACC: %7.03], [G LOSS: 173.75]
E: 5, [D ACC: %79.10], [D LOSS: 59.99], [G ACC: %2.73], [G LOSS: 215.24]
E: 6, [D ACC: %87.11], [D LOSS: 46.21], [G ACC: %10.16], [G LOSS: 194.84]
E: 7, [D ACC: %76.37], [D LOSS: 58.82], [G ACC: %13.28], [G LOSS: 175.04]
E: 8, [D ACC: %76.17], [D LOSS: 63.16], [G ACC: %16.02], [G LOSS: 161.22]
E: 9, [D ACC: %76.17], [D LOSS: 59.38], [G ACC: %13.28], [G LOSS: 160.77]
E: 10, [D ACC: %78.52], [D LOSS: 52.41], [G ACC: %8.98], [G LOSS: 184.13]
E: 11, [D ACC: %85.94], [D LOSS: 40.15], [G ACC: %7.03], [G LOSS: 215.76]
E: 12, [D ACC: %92.97], [D LOSS: 28.27], [G ACC: %7.03], [G LOSS: 247.75]
E: 13, [D ACC: %94.53], [D LOSS: 25.95], 

E: 111, [D ACC: %91.21], [D LOSS: 141.80], [G ACC: %0.00], [G LOSS: 1611.81]
E: 112, [D ACC: %92.77], [D LOSS: 116.63], [G ACC: %0.00], [G LOSS: 1611.81]
E: 113, [D ACC: %93.16], [D LOSS: 110.48], [G ACC: %0.00], [G LOSS: 1611.81]
E: 114, [D ACC: %93.36], [D LOSS: 107.24], [G ACC: %0.00], [G LOSS: 1611.81]
E: 115, [D ACC: %91.60], [D LOSS: 135.52], [G ACC: %0.00], [G LOSS: 1611.81]
E: 116, [D ACC: %91.99], [D LOSS: 129.24], [G ACC: %0.00], [G LOSS: 1611.81]
E: 117, [D ACC: %93.16], [D LOSS: 110.36], [G ACC: %0.00], [G LOSS: 1611.81]
E: 118, [D ACC: %90.04], [D LOSS: 160.63], [G ACC: %0.00], [G LOSS: 1611.81]
E: 119, [D ACC: %95.51], [D LOSS: 72.58], [G ACC: %0.00], [G LOSS: 1611.81]
E: 120, [D ACC: %90.62], [D LOSS: 151.18], [G ACC: %0.00], [G LOSS: 1611.81]
E: 121, [D ACC: %93.36], [D LOSS: 107.13], [G ACC: %0.00], [G LOSS: 1611.81]
E: 122, [D ACC: %88.87], [D LOSS: 179.49], [G ACC: %0.00], [G LOSS: 1611.81]
E: 123, [D ACC: %91.02], [D LOSS: 144.89], [G ACC: %0.00], [G LOSS: 1611.81]


E: 219, [D ACC: %100.00], [D LOSS: 0.13], [G ACC: %99.61], [G LOSS: 30.69]
E: 220, [D ACC: %100.00], [D LOSS: 0.20], [G ACC: %100.00], [G LOSS: 25.62]
E: 221, [D ACC: %100.00], [D LOSS: 0.25], [G ACC: %100.00], [G LOSS: 26.51]
E: 222, [D ACC: %100.00], [D LOSS: 0.24], [G ACC: %100.00], [G LOSS: 27.09]
E: 223, [D ACC: %100.00], [D LOSS: 0.08], [G ACC: %99.61], [G LOSS: 29.07]
E: 224, [D ACC: %100.00], [D LOSS: 0.10], [G ACC: %100.00], [G LOSS: 28.17]
E: 225, [D ACC: %100.00], [D LOSS: 0.15], [G ACC: %100.00], [G LOSS: 28.81]
E: 226, [D ACC: %100.00], [D LOSS: 0.07], [G ACC: %100.00], [G LOSS: 29.17]
E: 227, [D ACC: %100.00], [D LOSS: 0.15], [G ACC: %100.00], [G LOSS: 29.48]
E: 228, [D ACC: %100.00], [D LOSS: 0.08], [G ACC: %100.00], [G LOSS: 29.87]
E: 229, [D ACC: %100.00], [D LOSS: 0.16], [G ACC: %100.00], [G LOSS: 29.77]
E: 230, [D ACC: %100.00], [D LOSS: 0.09], [G ACC: %100.00], [G LOSS: 29.87]
E: 231, [D ACC: %100.00], [D LOSS: 0.08], [G ACC: %100.00], [G LOSS: 30.25]
E: 232, [D ACC

E: 329, [D ACC: %100.00], [D LOSS: 0.14], [G ACC: %0.00], [G LOSS: 618.10]
E: 330, [D ACC: %100.00], [D LOSS: 0.11], [G ACC: %0.00], [G LOSS: 631.16]
E: 331, [D ACC: %100.00], [D LOSS: 0.62], [G ACC: %0.00], [G LOSS: 590.41]
E: 332, [D ACC: %100.00], [D LOSS: 0.18], [G ACC: %0.00], [G LOSS: 591.64]
E: 333, [D ACC: %99.80], [D LOSS: 3.44], [G ACC: %0.00], [G LOSS: 591.18]
E: 334, [D ACC: %100.00], [D LOSS: 0.13], [G ACC: %0.00], [G LOSS: 602.90]
E: 335, [D ACC: %100.00], [D LOSS: 0.18], [G ACC: %0.00], [G LOSS: 613.18]
E: 336, [D ACC: %99.80], [D LOSS: 3.29], [G ACC: %0.00], [G LOSS: 625.70]
E: 337, [D ACC: %100.00], [D LOSS: 0.15], [G ACC: %0.00], [G LOSS: 634.29]
E: 338, [D ACC: %100.00], [D LOSS: 0.44], [G ACC: %0.00], [G LOSS: 609.68]
E: 339, [D ACC: %100.00], [D LOSS: 0.12], [G ACC: %0.00], [G LOSS: 616.71]
E: 340, [D ACC: %100.00], [D LOSS: 0.13], [G ACC: %0.00], [G LOSS: 628.72]
E: 341, [D ACC: %100.00], [D LOSS: 0.19], [G ACC: %0.00], [G LOSS: 633.47]
E: 342, [D ACC: %100.00], [

E: 439, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %0.00], [G LOSS: 890.82]


## Plot Loss

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

## Plot Accuracy

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();

In [ ]:
## save model
pth = os.path.join(models_path, 'gray-gan.h5')
gan.save(pth)

pth = os.path.join(models_path, 'gray-generator.h5')
generator.save(pth)

pth = os.path.join(models_path, 'gray-discriminator.h5')
discriminator.save(pth)